In [1]:
import glob
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from gfpgan import GFPGANer
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, ImageSequenceClip, CompositeAudioClip
import os
import googletrans
from googletrans import Translator
import tensorflow as tf
import numpy as np

EXTERNAL = False

if EXTERNAL:
  %store -r WIDTH
  %store -r HEIGHT
  %store -r TEXT
  %store -r VISUALIZE
  %store -r SAVE_FOLDER
  %store -r PROJECT_NAME
  %store -r SHOW_OUTPUT
  %store -r IMG_NUMBER
  %store -r THUMBNAIL
  %store -r SKIP_VIDEO
  %store -r MERGE_VIDEO
  %store -r TEXT_VOICE_GEN
  %store -r LANGUAGES
  %store -r GEN_VIDEO
  %store -r INSERT_BACKGROUND
  %store -r RECORD_FRAME_INTER_AFTER
  %store -r RECORD_FRAME_INTER_HD_AFTER
else:
  WIDTH = 768
  HEIGHT = 768
  VISUALIZE = True  
  SAVE_FOLDER = "D:\\Deletar\\p_gen"
  PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
  STYLES_FOLDER = ".\\styles"
  SHOW_OUTPUT = True
  IMG_NUMBER = 2
  SKIP_VIDEO = []
  MERGE_VIDEO = False
  TEXT_VOICE_GEN = ['A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A']
  LANGUAGES = ['pt']
  GEN_VIDEO = True
  INSERT_BACKGROUND = False
  RECORD_FRAME_INTER_AFTER = 0
  RECORD_FRAME_INTER_HD_AFTER = 0

ANIME = False
translator = Translator()

In [2]:

project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"

netscale = 4
model_path_face = "./realesrgan/GFPGANv1.3.pth"
dni_weight = None
tile = 0
title_pad = 10
pre_pad = 0
half = None
gpu_id = None
upscale = 3.5
upscale = 4
img_mode = 'RGBA'

if ANIME:
  # R-ESRGAN + Anime
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus_anime_6B.pth"
    # R-ESRGAN
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus.pth"
  upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path_x4,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
  )

  face_enhancer = GFPGANer(
            model_path=model_path_face,
            upscale=upscale,
            arch='clean',
            channel_multiplier=2,
            bg_upsampler=upsampler)
else:
  from basicsr.archs.rrdbnet_arch import RRDBNet
  arch = 'clean'
  channel_multiplier = 2
  model_name = 'GFPGANv1.4'
  url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'
  model_path = os.path.join('experiments/pretrained_models', model_name + '.pth')
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
  bg_upsampler = RealESRGANer(
            scale=2,
            model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
            model=model,
            tile=400,
            tile_pad=10,
            pre_pad=0,
            half=True)  # need to set False in CPU mode
  



  restorer = GFPGANer(
    model_path=model_path,
    upscale=upscale,
    arch=arch,
    channel_multiplier=channel_multiplier,
    bg_upsampler=bg_upsampler)




def resize_image(data):
  resized_img = tf.image.resize(
  images=data,
  size=[270, 480],   
  method=tf.image.ResizeMethod.BILINEAR,
  preserve_aspect_ratio=False,
  antialias=True,
  )    
  return resized_img

c:\Users\Matheus\anaconda3\envs\Pyflow\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Matheus\anaconda3\envs\Pyflow\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
#create srt
""" def create_srt(text, initial_time, final_time, indx, filename):
  initial_mil = int(round(initial_time % 1,3)*1000)
  final_mil = int(round(final_time % 1,3)*1000)
  initial_time = int(initial_time)
  final_time = int(final_time)

  minutes, seconds = divmod(initial_time, 60)
  hours, minutes = divmod(minutes, 60)
  initial_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{initial_mil}'

  minutes, seconds = divmod(final_time, 60)
  hours, minutes = divmod(minutes, 60)
  final_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{final_mil}'

  with open(f"{filename}//en.srt" , 'w', encoding='utf-8') as f:
    f.write(f'{int(indx)+1}\n')
    f.write(f'{initial_time} --> {final_time}\n')
    f.write(f'{text}\n')
    f.write('\n')
  
  for lang in LANGUAGES:
    transl = translator.translate(text, src='en', dest=lang).text
    with open(f"{filename}//{lang}.srt" , 'w', encoding='utf-8') as f:
      f.write(f'{int(indx)+1}\n')
      f.write(f'{initial_time} --> {final_time}\n')
      f.write(f'{transl}\n')
      f.write('\n')

def merge_srt_files(filenames, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        for i, filename in enumerate(filenames):
            with open(filename, 'r', encoding='utf-8') as infile:
                for line in infile:
                    outfile.write(line)
                if i < len(filenames) - 1:
                    outfile.write('\n') """

' def create_srt(text, initial_time, final_time, indx, filename):\n  initial_mil = int(round(initial_time % 1,3)*1000)\n  final_mil = int(round(final_time % 1,3)*1000)\n  initial_time = int(initial_time)\n  final_time = int(final_time)\n\n  minutes, seconds = divmod(initial_time, 60)\n  hours, minutes = divmod(minutes, 60)\n  initial_time = f\'{hours:02d}:{minutes:02d}:{seconds:02d},{initial_mil}\'\n\n  minutes, seconds = divmod(final_time, 60)\n  hours, minutes = divmod(minutes, 60)\n  final_time = f\'{hours:02d}:{minutes:02d}:{seconds:02d},{final_mil}\'\n\n  with open(f"{filename}//en.srt" , \'w\', encoding=\'utf-8\') as f:\n    f.write(f\'{int(indx)+1}\n\')\n    f.write(f\'{initial_time} --> {final_time}\n\')\n    f.write(f\'{text}\n\')\n    f.write(\'\n\')\n  \n  for lang in LANGUAGES:\n    transl = translator.translate(text, src=\'en\', dest=lang).text\n    with open(f"{filename}//{lang}.srt" , \'w\', encoding=\'utf-8\') as f:\n      f.write(f\'{int(indx)+1}\n\')\n      f.write(f\

In [5]:
print('start')
if EXTERNAL and MERGE_VIDEO:
    video = []
    cont = -1
    start_time = 0    
    """ filenames = []
    filenames_other = {}
    for lan in LANGUAGES:
        filenames_other[lan] = [] """
    
    if (not os.path.exists(f"{project_folder}//video_sound_hd")):
                    os.makedirs(f"{project_folder}//video_sound_hd")
    for folder in os.listdir(f"{project_folder}//video_sound"):
        for filename in os.listdir(f"{project_folder}//video_sound//{folder}"):
            if (filename[-3:] == 'mp4'):
                cont = cont + 1
                if (RECORD_FRAME_INTER_HD_AFTER < cont):
                    print(filename[:-4])
                    frames_hd = []
                    addr_video = f"{project_folder}//video_sound//{folder}//{filename}"
                    video_lq = VideoFileClip(addr_video)                    
                    original_audio = video_lq.audio
                    original_audio_duration = round(original_audio.duration,3)
                    for i, frame in enumerate(video_lq.iter_frames()):
                        print(f'Frame: {i}')
                        if ANIME:                    
                            _, _, output = face_enhancer.enhance(resize_image(frame).numpy(), has_aligned=False, only_center_face=False, paste_back=True)
                        else:
                            _, _, restored_img = restorer.enhance(
                                frame,
                                has_aligned=False,
                                only_center_face=False,
                                paste_back=True,
                                weight=0.5
                            )
                        frames_hd.append(output)
                    if (not os.path.exists(f"{project_folder}//video_sound_hd//{filename[:-4]}")):
                        os.makedirs(f"{project_folder}//video_sound_hd//{filename[:-4]}")
                    video_hd = ImageSequenceClip(frames_hd, fps=30)
                    video_hd = video_hd.set_audio(original_audio)
                    video_hd_duration = round(video_hd.duration,3)
                    # create_srt(TEXT_VOICE_GEN[cont], start_time + 0.5, start_time + original_audio_duration, cont, filename=f"{project_folder}//video_sound//{cont:07d}")
                    # start_time = round(start_time + video_hd_duration,3)
                    if os.path.exists(f"{project_folder}//video_sound_hd//{filename[:-4]}//{filename}"):
                        os.remove(f"{project_folder}//video_sound_hd//{filename[:-4]}//{filename}")  
                    video_hd.write_videofile(f"{project_folder}//video_sound_hd//{filename[:-4]}//{filename}")
    for folder in os.listdir(f"{project_folder}//video_sound_hd"):
        for filename in os.listdir(f"{project_folder}//video_sound_hd//{folder}"):                   
            if (filename[-3:] == 'mp4'):
                addr_video = f"{project_folder}//video_sound_hd//{folder}//{filename}"                
                video_clip = VideoFileClip(addr_video)                     
                video.append(video_clip)
            
    """ for folder in os.listdir(f"{project_folder}//video_sound"):
        for filename in os.listdir(f"{project_folder}//video_sound//{folder}"):
            if (filename[-6:] == 'en.srt'):                    
                filenames.append(f"{project_folder}//video_sound//{folder}//{filename}")
            elif(filename[-3:] == 'srt'):
                lan = filename[-6:-4]
                filenames_other[lan] = filenames_other[lan] + [f"{project_folder}//video_sound//{folder}//{filename}"]
    
    if len(filenames) > 0:
        merge_srt_files(filenames, f"{project_folder}//full//full.srt")
        for lang_mod in filenames_other:
            merge_srt_files(filenames_other[lang_mod], f"{project_folder}//full//{lang_mod}.srt") """
                
    
    merged_video = concatenate_videoclips(video, method='chain')
    merged_video.write_videofile(f"{project_folder}//full//full_hd.mp4")
else:
    frames_hd = []
    addr_video = ".\\result2.mp4"
    video_lq = VideoFileClip(addr_video)                    
    original_audio = video_lq.audio
    original_audio_duration = round(video_lq.duration,3)
    for i, frame in enumerate(video_lq.iter_frames()):                    
        _, _, output = face_enhancer.enhance(resize_image(frame).numpy(), has_aligned=False, only_center_face=False, paste_back=True)
        # output, _ = upsampler.enhance(frame)
        frames_hd.append(output)
    video_hd = ImageSequenceClip(frames_hd, fps=30)
    video_hd = video_hd.set_audio(original_audio)
    video_hd_duration = round(video_hd.duration,3)
    video_hd.write_videofile(".\\final_hd.mp4")

Moviepy - Building video .\final_hd.mp4.
Moviepy - Writing video .\final_hd.mp4



Moviepy - Done !
Moviepy - video ready .\final_hd.mp4
